# Importing necessary libraries

In [1]:
!pip uninstall -y torch torchvision torchaudio transformers peft datasets
!pip cache purge

Found existing installation: torch 2.8.0
Uninstalling torch-2.8.0:
  Successfully uninstalled torch-2.8.0
Found existing installation: transformers 4.55.4
Uninstalling transformers-4.55.4:
  Successfully uninstalled transformers-4.55.4
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
Found existing installation: datasets 2.21.0
Uninstalling datasets-2.21.0:
  Successfully uninstalled datasets-2.21.0
Files removed: 38 (902.1 MB)


In [3]:
!pip uninstall -y torch
!pip install torch==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 29.5 MB/s  0:00:11m0:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bert-score 0.3.13 requires transformers>=3.0.0, which is not installed.
litdata 0.2.50 requires torchvision, which is not installed.


In [4]:
!pip install --upgrade transformers==4.55.4 peft==0.17.1 "datasets[audio]>=2.21.0" accelerate evaluate jiwer tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [datasets]3/4 [datasets]ers]


In [5]:
!pip uninstall -y datasets torchcodec soundfile
!pip install "datasets[audio]==2.21.0" --no-cache-dir

Found existing installation: datasets 4.1.1
Uninstalling datasets-4.1.1:
  Successfully uninstalled datasets-4.1.1
Found existing installation: torchcodec 0.7.0
Uninstalling torchcodec-0.7.0:
  Successfully uninstalled torchcodec-0.7.0
Found existing installation: soundfile 0.13.1
Uninstalling soundfile-0.13.1:
  Successfully uninstalled soundfile-0.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 28.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [datasets]1/2 [datasets]


In [6]:
!pip install bert-score
!pip install SentencePiece

# Preparing dataset

In [7]:
import zipfile

zip_path1 = '/teamspace/studios/this_studio/en-hi.txt.zip'
zip_path2 = '/teamspace/studios/this_studio/en-hi.txt (3).zip'
extract_dir1 = '/teamspace/studios/this_studio/corpus/opensub'
extract_dir2 = '/teamspace/studios/this_studio/corpus/anuvaad'

with zipfile.ZipFile(zip_path1, 'r') as zip_ref:
    zip_ref.extractall(extract_dir1)

with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
    zip_ref.extractall(extract_dir2)


In [7]:
with open('/teamspace/studios/this_studio/corpus/opensub/OpenSubtitles.en-hi.en','r') as file1:
  en_lines = file1.readlines()
with open('/teamspace/studios/this_studio/corpus/opensub/OpenSubtitles.en-hi.hi','r') as file2:
  hi_lines = file2.readlines()

with open('/teamspace/studios/this_studio/corpus/anuvaad/Anuvaad.en-hi.en','r') as file1:
  en_lines1 = file1.readlines()
with open('/teamspace/studios/this_studio/corpus/anuvaad/Anuvaad.en-hi.hi','r') as file2:
  hi_lines1 = file2.readlines()

In [8]:
print(len(en_lines))
print(len(en_lines1))

3004537
5029374


In [9]:
en_lines.extend(en_lines1)
hi_lines.extend(hi_lines1)

In [10]:
from datasets import Dataset

dict1 = {"en":en_lines,"hi":hi_lines}

ds = Dataset.from_dict(dict1)

In [11]:
# Split the dataset into a temporary set and a test set (e.g., 90% and 10%)
train_test_split = ds.train_test_split(test_size=0.1, seed=42)

# Access the train and test splits
active_dataset = train_test_split['test']

In [12]:
# Split the dataset into a temporary set and a test set (e.g., 90% and 10%)
train_test_split = active_dataset.train_test_split(test_size=0.18, seed=42)

# Access the train and test splits
train_dataset = train_test_split['train']
temp_dataset = train_test_split['test']

train_test_split = temp_dataset.train_test_split(test_size=0.5, seed=42)

val_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [2]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

NameError: name 'train_dataset' is not defined

# Loading model and tokenizer

In [14]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, NllbTokenizer
from peft import prepare_model_for_kbit_training

tokenizer = NllbTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn", # English
    tgt_lang="hin_Deva"  # Hindi
)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

model = prepare_model_for_kbit_training(model)

In [15]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

# Apply the PEFT configuration to the model
model = get_peft_model(model, lora_config)

# Print the number of trainable parameters
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 619,792,384 || trainable%: 0.7613


# Mapping of data

In [16]:
def filter_empty_strings_and_none(example):
    # Check for None values
    en_text = example.get("en")
    hi_text = example.get("hi")
    
    if en_text is None or hi_text is None:
        return False
        
    # Check for empty strings after stripping whitespace
    return en_text.strip() != "" and hi_text.strip() != ""

    

In [17]:
train_filtered = train_dataset.filter(filter_empty_strings_and_none)
val_filtered = val_dataset.filter(filter_empty_strings_and_none)
test_filtered = test_dataset.filter(filter_empty_strings_and_none)

Filter:   0%|          | 0/2832757 [00:00<?, ? examples/s]

Filter:   0%|          | 0/310912 [00:00<?, ? examples/s]

Filter:   0%|          | 0/310913 [00:00<?, ? examples/s]

In [18]:
import torch

def prepare_dataset(example):
    # Safely get the values, handling potential None or missing keys
    input_text = example.get("en", "")
    output_text = example.get("hi", "")

    # Ensure the values are strings before stripping
    input_text = str(input_text).strip()
    output_text = str(output_text).strip()

    # Tokenize the input text
    # We do not use padding here as it is handled more efficiently by a DataCollator
    input_encodings = tokenizer(
        input_text,
        max_length=512,
        truncation=True
    )

    # Tokenize the target text within the as_target_tokenizer() context
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(
            output_text,
            max_length=512,
            truncation=True
        )

    # Return the dictionary with token IDs and attention mask
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [19]:
train_mapped = train_filtered.map(prepare_dataset)
val_mapped = val_filtered.map(prepare_dataset)
test_mapped = test_filtered.map(prepare_dataset)

Map:   0%|          | 0/2832722 [00:00<?, ? examples/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/310910 [00:00<?, ? examples/s]

Map:   0%|          | 0/310909 [00:00<?, ? examples/s]

# Data Collator

In [20]:
from transformers import DataCollatorForSeq2Seq

# Assuming 'tokenizer' is your NLLB tokenizer and 'model' is your NLLB model
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    label_pad_token_id=tokenizer.pad_token_id  # Use the tokenizer's pad token
)

# Creating Trainer

In [29]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./nllb-finetuned",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_steps=8700,  # To finish training in ~3 hours
    logging_steps=500,
    eval_steps=500, # Evaluate more frequently to monitor progress
    do_train=True,
    do_eval=True,
    predict_with_generate=True,
    save_total_limit=1,
    report_to="none",
    fp16=True,
    dataloader_num_workers=8,
    per_device_eval_batch_size=16,
)

In [25]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_mapped,
    eval_dataset=val_mapped,
    data_collator=data_collator,
    tokenizer=tokenizer, 
)

/tmp/ipykernel_4347/837170514.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Training process

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"  #wandB is used for checking metrics and all, which we don't necessarily need here, without an api key you can't run this

trainer.train()

NameError: name 'trainer' is not defined

# Saving Model and Downloading it

In [ ]:
# Assuming your fine-tuned PEFT model is named 'model'
# and your Trainer object is named 'trainer'
import os

# Define the directory where you want to save the adapter
# This is typically the same as your Trainer's output directory
adapter_dir = "./nllb-finetuned-en-hi"
os.makedirs(adapter_dir, exist_ok=True)

# Save the adapter weights
# This will create 'adapter_model.bin' and 'adapter_config.json' in the directory
model.save_pretrained(adapter_dir)

In [ ]:
import zipfile

def zip_directory(directory_to_zip, zip_file_path):
    """Creates a zip file from a directory."""
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory_to_zip):
            for file in files:
                # Create the relative path to put files at the zip's root
                relative_path = os.path.relpath(os.path.join(root, file), directory_to_zip)
                zipf.write(os.path.join(root, file), relative_path)

# Define the paths
adapter_dir = "./nllb-finetuned-en-hi"
zip_file_path = f"{adapter_dir}.zip"

# Create the zip file
zip_directory(adapter_dir, zip_file_path)

print(f"Zip file created at: {zip_file_path}")